<a href="https://colab.research.google.com/github/Ali-Kazmi/All-Pairs-Shortest-Path/blob/master/APSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will use python's networkX library to perform All Pairs Shortest Paths (APSP) on the california road dataset. Then, i'll use scipy to do the same much faster. Lastly, I'll implement my own and use numba to speed it up, beating scipy! 

In [2]:
# Get the data 
from pandas import read_csv
OLcnode = read_csv('https://www.cs.utah.edu/~lifeifei/research/tpq/OL.cnode',
                 names=['Node_ID', 'X','Y'],
                 delim_whitespace=True, nrows=5000) 

OLcedge=read_csv('https://www.cs.utah.edu/~lifeifei/research/tpq/OL.cedge',
                 names=['Edge ID', 'Start_Node_ID','End_Node_ID','L2_distance'],
                 delim_whitespace=True,nrows=5000) 


In [35]:
OLcedge.head() 

,Edge ID,Start_Node_ID,End_Node_ID,L2_distance
0,0,1609,1622,57.403187
1,1,2471,2479,29.718756
2,2,2463,2471,61.706902
3,3,2443,2448,19.080025
4,4,1417,1491,28.248583


In [ ]:
OLcnode.head()

,Node_ID,X,Y
0,0,769.948669,2982.984131
1,1,863.275757,3005.275635
2,2,690.196411,3333.704834
3,3,1197.556519,2984.470215
4,4,1261.188599,2985.956299


The cell below will output an example of one shortest path (between two vertices). For APSP, we will be finding these shortest paths for all shortest paths between all pairs of vertices. As you can imagine, it gets expensive fast 

In [3]:
def get_edgelist(df):
    ### BEGIN SOLUTION
    return [(a, b, {'w': w}) for a, b, w in zip(df["Start_Node_ID"], df["End_Node_ID"], df["L2_distance"])]
    ### END SOLUTION
    
# Demo
edgelist = get_edgelist(OLcedge)
edgelist[:5] #just display a few. There's a lot of these! 

[(1609, 1622, {'w': 57.403187}),
 (2471, 2479, {'w': 29.718756}),
 (2463, 2471, {'w': 61.706902}),
 (2443, 2448, {'w': 19.080025}),
 (1417, 1491, {'w': 28.248583})]

In [4]:

from networkx import Graph
G = Graph()
G.add_nodes_from(OLcnode["Node_ID"])
G.add_edges_from(edgelist)

print(f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

The graph has 5590 nodes and 4995 edges.


In [ ]:
def get_shortest_path(G):
    from networkx import all_pairs_dijkstra_path
    return dict(all_pairs_dijkstra_path(G, weight='w'))
start = OLcnode["Node_ID"].iloc[1]
finish = OLcnode["Node_ID"].iloc[3]
#print(f"Calculating a shortest path between all pairs of nodes")
%timeit -n 3 get_shortest_path(G)
#print(f"\n==> This cell made a dictionary of shortest paths using networkX `{type(path)}`:")
#print("To use, enter two nodeID's and this will return the shortest path.")
#print("If there is no path,it will throw a key error as the pairs that cannot be reached are not stored.")
path1 = get_shortest_path(G)
#print(path[start][finish])

3 loops, best of 3: 14.5 s per loop


In [ ]:
def get_shortest_path(G):
    from networkx import all_pairs_bellman_ford_path
    return dict(all_pairs_bellman_ford_path(G, weight='w'))
start = OLcnode["Node_ID"].iloc[1]
finish = OLcnode["Node_ID"].iloc[3]
#print(f"Calculating a shortest path between all pairs of nodes")
%timeit -n 3 get_shortest_path(G)
#print(f"\n==> This cell made a dictionary of shortest paths using networkX `{type(path)}`:")
#print("To use, enter two nodeID's and this will return the shortest path.")
#print("If there is no path,it will throw a key error as the pairs that cannot be reached are not stored.")
path = get_shortest_path(G) #Doesn't effect time
#print(path[start][finish])

3 loops, best of 3: 5.96 s per loop


In [ ]:
print(path[1][5])

[1, 0, 2, 5]
time: 1.55 ms


In [ ]:
from networkx import floyd_warshall

%timeit floyd_warshall(G)


KeyboardInterrupt: ignored

In [ ]:
from networkx import floyd_warshall_numpy
%timeit -n 3 floyd_warshall_numpy(G)

pathfwnp = floyd_warshall_numpy(G)
#Much better, only 18-24 seconds on a 1500 or so node graph

3 loops, best of 3: 4min 51s per loop


In [ ]:
pathfwnp #Note: I only printed this for the smaller graph, the 1695 node one. This is just to give you an idea of the output 

matrix([[ 0.,  1.,  1., ..., inf, inf, inf],
        [ 1.,  0.,  2., ..., inf, inf, inf],
        [ 1.,  2.,  0., ..., inf, inf, inf],
        ...,
        [inf, inf, inf, ...,  0.,  2.,  3.],
        [inf, inf, inf, ...,  2.,  0.,  1.],
        [inf, inf, inf, ...,  3.,  1.,  0.]])

time: 3.93 ms


Now, I'm going to do the following: 

1) convert the networkx graph we had above into an adjacency matrix representation in numpy 

2) use SciPY shortest path on the new adjacency matrix, to test the speed of a numpy based approach 

3) use numba to speed this up? (edit: numba can't just speed up scipy) 

> Indented block



In [5]:
import numpy as np
from networkx import to_numpy_array
adjlist = to_numpy_array(G)

In [ ]:
print(adjlist.shape)
adjlist.view()


(74, 74)


array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import scipy as sp
import scipy.sparse 

%timeit sp.sparse.csgraph.shortest_path(adjlist,method='FW')

1 loop, best of 3: 376 ms per loop



Looking into how Scipy does it https://github.com/scipy/scipy/blob/master/scipy/sparse/csgraph/_shortest_path.pyx, the trick is using cython (it compiles to C) and numpy. It also takes advantage of the sparsity of the problem. 

In [ ]:
result.view()

array([[ 0.,  1.,  1., ..., 73., inf, inf],
       [ 1.,  0.,  2., ..., 72., inf, inf],
       [ 1.,  2.,  0., ..., 74., inf, inf],
       ...,
       [73., 72., 74., ...,  0., inf, inf],
       [inf, inf, inf, ..., inf,  0.,  1.],
       [inf, inf, inf, ..., inf,  1.,  0.]])

time: 4.45 ms


In [ ]:
adjlist = to_numpy_array(G)

In [ ]:
import numpy as np 

def floyd_warshall_simple(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num):
    for i in range(vertex_num):
      for j in range(vertex_num):
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [ ]:
%timeit -n 3 floyd_warshall_simple(adjlist,4228)

KeyboardInterrupt: ignored

In [ ]:
#Correctness checks 
def test_floyd_warshall_algorithms_on_small_matrix():
    INPUT = array([
        [  0.,  inf,  -2.,  inf],
        [  4.,   0.,   3.,  inf],
        [ inf,  inf,   0.,   2.],
        [ inf,  -1.,  inf,   0.]
    ])

    OUTPUT = array([
        [ 0., -1., -2.,  0.],
        [ 4.,  0.,  2.,  4.],
        [ 5.,  1.,  0.,  2.],
        [ 3., -1.,  1.,  0.]])
    print(OUTPUT)
    print(floyd_warshall_simple(INPUT,4))
    #assert (floyd_warshall_naive(INPUT) == OUTPUT).all()
test_floyd_warshall_algorithms_on_small_matrix()

[[ 0. -1. -2.  0.]
 [ 4.  0.  2.  4.]
 [ 5.  1.  0.  2.]
 [ 3. -1.  1.  0.]]
[[ 0. -1. -2.  0.]
 [ 4.  0.  2.  4.]
 [ 5.  1.  0.  2.]
 [ 3. -1.  1.  0.]]
time: 9.49 ms


In [ ]:
!pip install numba

In [ ]:
from numba import jit
import numpy as np 

@jit(nopython=True)
def floyd_warshall_nb(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in range(vertex_num): #i, j is indep, prange >
      for j in range(vertex_num): #i, j is indep,  prange?
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [ ]:
adjlist=to_numpy_array(G)

On the first call for the graph with 1700x1700 adj matrix, it takes 662 ms. THis is because the first time the function is run it has to do numba's compile step. Second call and onwards takes in the order of 3.62 ms to 6.5 ms. TODO: run this experiment on bigger graphs, as at this speed other costs can come into play.  

In [ ]:
obj= %timeit -n 3 -o floyd_warshall_nb(adjlist,4228)
obj.all_runs

3 loops, best of 3: 2min 9s per loop


[388.97960488699937, 388.3968831210004, 388.8255705519987]

Ok, since this is using numba let's see how it looks below the hood. 

Just by adding jit we took down the runtime down drastically (from 1.7 seconds to 404 ms. Now our implementation is faster than scipys by a significant amount. But, let's parallelize ours! 

In [ ]:
#Now, to parallelize it 
from numba import jit
from numba import prange
import numpy as np 

@jit(nopython=True,parallel=True)
def floyd_warshall_parallel_nb(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in prange(vertex_num):
    dist[a][a]=0
  for b in prange(vertex_num):
    for c in prange(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in prange(vertex_num): #i, j is indep, prange to parallize it
      for j in prange(vertex_num): #i, j is indep,  prange to parallelize it 
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [ ]:
obj= %timeit -n 1 -o floyd_warshall_parallel_nb(adjlist,4228)
obj.all_runs

1 loop, best of 3: 1min 32s per loop


[94.98123240099994, 93.3153924659996, 92.84180907300015]

In [ ]:
from numba import jit
from numba import prange
import numpy as np 

@jit(nopython=True,parallel=True,fastmath=True)
def floyd_warshall_parallel_nb_optimized(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in prange(vertex_num):
    dist[a][a]=0
  for b in prange(vertex_num):
    for c in prange(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in prange(vertex_num): #i, j is indep, prange to parallize it
      for j in prange(vertex_num): #i, j is indep,  prange to parallelize it 
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [ ]:
obj= %timeit -n 1 -o floyd_warshall_parallel_nb_optimized(adjlist,74)
obj.all_runs

The slowest run took 675.74 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 2.68 ms per loop


[1.8132510680006817, 0.009130424000431958, 0.0026833379997697193]

In [ ]:
from numba import jit
from numba import prange
import numpy as np 

@jit(nopython=True,parallel=True)
def floyd_warshall_parallel_nb_opt(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in prange(vertex_num):
    dist[a][a]=0
  for b in prange(vertex_num):
    for c in prange(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in prange(vertex_num): #i, j is indep, prange to parallize it
      for j in prange(vertex_num): #i, j is indep,  prange to parallelize it 
        myint=dist[i][k]+dist[k][j]
        if dist[i][j] > myint:
          dist[i][j] = myint
  return dist

In [ ]:
obj= %timeit -n 1 -o floyd_warshall_parallel_nb_opt(adjlist,74)
obj.all_runs

The slowest run took 792.46 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 2.38 ms per loop


[1.8849546050005301, 0.002588338999885309, 0.00237860300057946]

In [ ]:
#This is to implement it on a semiring with the min + operations. 
import numpy as np 
@jit(nopython=True,parallel=True)
def floyd_warshall_semir(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num):
    for i in prange(vertex_num):
      for j in prange(vertex_num):
        dist[i][j]=min(dist[i][j],dist[i][k]+dist[k][j])
  return dist
  #Todo: correctness check and benchmarks 

NameError: ignored

In [ ]:
obj= %timeit -n 1 -o floyd_warshall_semir(adjlist,4228)
obj.all_runs

1 loop, best of 3: 1min 11s per loop


[72.53005440700144, 71.52838812999835, 71.8490746690004]

In [7]:
%load_ext Cython

In [ ]:
%%cython
import numpy as np 
import numba

def floyd_warshall_cython(graph, int vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  cdef int a,b,c,k,j,i
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num):
    for i in range(vertex_num):
      for j in range(vertex_num):
          dist[i][j]=min(dist[i][j],dist[i][k]+dist[k][j])
  return dist

In [ ]:
obj= %timeit -n 1 -o floyd_warshall_cython(adjlist,74)
obj.all_runs
#todo try small graph (by hand) for correctness (are my types right?) 
#wait why would i be trying to add infinity??? need an if else somewhere
#todo: cython prange???
#double check correctness (if same answer but slower, try to track it down) (check the core+scipy scaffolding) (try to get scipys running for me)
#another thing to try: not colab! Does this happen on jupyter local???? If that doesn't work, try stackoverflow post (probalby figure it out in the proces)
#after this is working, introduce parallelism (goal: parallel numba/scipy > scipy)
# why is there a gap between numab and cython? 
#next: parent pointers! Algo variations (how to make it work better)
# next: APSP once then the graph changes. Is there a way to update and reuse instead of recompute (there has to be smt to do this...)
# what does the updating look like in algbraic? Should look similar? Need to check (2 cases: #nodes is the same, only edges change (matrix a+ delta + a times delta))

1 loop, best of 3: 315 ms per loop


[0.3347447630003444, 0.31512190400007967, 0.33910887599995476]

In [8]:
#I took the source code of scipy, trimmed it down extensively, and put it here. 
#It took some work figuring out what to do with the types; they used a PXI file and included it originally
#Instead, I just put them at the top of the cell (this is the Dtype stuff)
%%cython
import warnings

import numpy as np
cimport numpy as np

from scipy.sparse import csr_matrix, isspmatrix, isspmatrix_csr, isspmatrix_csc
from scipy.sparse.csgraph._validation import validate_graph

cimport cython

from libc.stdlib cimport malloc, free
from numpy.math cimport INFINITY

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

ITYPE = np.int32
ctypedef np.int32_t ITYPE_t

# Fused type for int32 and int64
ctypedef fused int32_or_int64:
    np.int32_t
    np.int64_t

# EPS is the precision of DTYPE
DEF DTYPE_EPS = 1E-15

# NULL_IDX is the index used in predecessor matrices to store a non-path
DEF NULL_IDX = -9999

class NegativeCycleError(Exception):
    def __init__(self, message=''):
        Exception.__init__(self, message)

def floyd_warshall_scipy(csgraph, directed=True,
                   return_predecessors=False,
                   unweighted=False,
                   overwrite=False):
    dist_matrix = validate_graph(csgraph, directed, DTYPE,
                                 csr_output=False,
                                 copy_if_dense=not overwrite)
    if not isspmatrix(csgraph):
        # for dense array input, zero entries represent non-edge
        dist_matrix[dist_matrix == 0] = INFINITY

    if unweighted:
        dist_matrix[~np.isinf(dist_matrix)] = 1

    if return_predecessors:
        predecessor_matrix = np.empty(dist_matrix.shape,
                                      dtype=ITYPE, order='C')
    else:
        predecessor_matrix = np.empty((0, 0), dtype=ITYPE)

    _floyd_warshall(dist_matrix,
                    predecessor_matrix,
                    int(directed))

    if np.any(dist_matrix.diagonal() < 0):
        raise NegativeCycleError("Negative cycle in nodes %s"
                                 % np.where(dist_matrix.diagonal() < 0)[0])

    if return_predecessors:
        return dist_matrix, predecessor_matrix
    else:
        return dist_matrix

@cython.boundscheck(False)
cdef void _floyd_warshall(
               np.ndarray[DTYPE_t, ndim=2, mode='c'] dist_matrix,
               np.ndarray[ITYPE_t, ndim=2, mode='c'] predecessor_matrix,
               int directed=0):
    # dist_matrix : in/out
    #    on input, the graph
    #    on output, the matrix of shortest paths
    # dist_matrix should be a [N,N] matrix, such that dist_matrix[i, j]
    # is the distance from point i to point j.  Zero-distances imply that
    # the points are not connected.
    cdef int N = dist_matrix.shape[0]
    assert dist_matrix.shape[1] == N

    cdef unsigned int i, j, k

    cdef DTYPE_t d_ijk

    # ----------------------------------------------------------------------
    #  Initialize distance matrix
    #   - set diagonal to zero
    #   - symmetrize matrix if non-directed graph is desired
    dist_matrix.flat[::N + 1] = 0
    if not directed:
        for i in range(N):
            for j in range(i + 1, N):
                if dist_matrix[j, i] <= dist_matrix[i, j]:
                    dist_matrix[i, j] = dist_matrix[j, i]
                else:
                    dist_matrix[j, i] = dist_matrix[i, j]

    #----------------------------------------------------------------------
    #  Initialize predecessor matrix
    #   - check matrix size
    #   - initialize diagonal and all non-edges to NULL
    #   - initialize all edges to the row index
    cdef int store_predecessors = False

    if predecessor_matrix.size > 0:
        store_predecessors = True
        assert predecessor_matrix.shape[0] == N
        assert predecessor_matrix.shape[1] == N
        predecessor_matrix.fill(NULL_IDX)
        i_edge = np.where(~np.isinf(dist_matrix))
        predecessor_matrix[i_edge] = i_edge[0]
        predecessor_matrix.flat[::N + 1] = NULL_IDX

    # Now perform the Floyd-Warshall algorithm.
    # In each loop, this finds the shortest path from point i
    #  to point j using intermediate nodes 0 ... k
    if store_predecessors:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
                        predecessor_matrix[i, j] = predecessor_matrix[k, j]
    else:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk

In [85]:
%timeit floyd_warshall_scipy(adjlist)

1 loop, best of 3: 58.8 s per loop


In [9]:
#I took the source code of scipy, trimmed it down extensively, and put it here. 
#It took some work figuring out what to do with the types; they used a PXI file and included it originally
#Instead, I just put them at the top of the cell (this is the Dtype stuff)
#%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
%%cython -f
import warnings
from cython.parallel import prange
import cython.parallel as cp
import numpy as np
cimport numpy as np
cimport openmp
from scipy.sparse import csr_matrix, isspmatrix, isspmatrix_csr, isspmatrix_csc
from scipy.sparse.csgraph._validation import validate_graph

cimport cython

from libc.stdlib cimport malloc, free
from numpy.math cimport INFINITY

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

ITYPE = np.int32
ctypedef np.int32_t ITYPE_t

# Fused type for int32 and int64
ctypedef fused int32_or_int64:
    np.int32_t
    np.int64_t

# EPS is the precision of DTYPE
DEF DTYPE_EPS = 1E-15

# NULL_IDX is the index used in predecessor matrices to store a non-path
DEF NULL_IDX = -9999


class NegativeCycleError(Exception):
    def __init__(self, message=''):
        Exception.__init__(self, message)

def floyd_warshall_scipy_parallel(csgraph, directed=True,
                   return_predecessors=False,
                   unweighted=False,
                   overwrite=False):
    dist_matrix = validate_graph(csgraph, directed, DTYPE,
                                 csr_output=False,
                                 copy_if_dense=not overwrite)
    if not isspmatrix(csgraph):
        # for dense array input, zero entries represent non-edge
        dist_matrix[dist_matrix == 0] = INFINITY

    if unweighted:
        dist_matrix[~np.isinf(dist_matrix)] = 1

    if return_predecessors:
        predecessor_matrix = np.empty(dist_matrix.shape,
                                      dtype=ITYPE, order='C')
    else:
        predecessor_matrix = np.empty((0, 0), dtype=ITYPE)

    _floyd_warshall(dist_matrix,
                    predecessor_matrix,
                    int(directed))

    if np.any(dist_matrix.diagonal() < 0):
        raise NegativeCycleError("Negative cycle in nodes %s"
                                 % np.where(dist_matrix.diagonal() < 0)[0])

    if return_predecessors:
        return dist_matrix, predecessor_matrix
    else:
        return dist_matrix

@cython.boundscheck(False)
cdef void _floyd_warshall(
               np.ndarray[DTYPE_t, ndim=2, mode='c'] dist_matrix,
               np.ndarray[ITYPE_t, ndim=2, mode='c'] predecessor_matrix,
               int directed=0):
    # dist_matrix : in/out
    #    on input, the graph
    #    on output, the matrix of shortest paths
    # dist_matrix should be a [N,N] matrix, such that dist_matrix[i, j]
    # is the distance from point i to point j.  Zero-distances imply that
    # the points are not connected.
    cdef int N = dist_matrix.shape[0]
    assert dist_matrix.shape[1] == N

    #cdef unsigned int i, j, k
    cdef int i, j, k
    cdef DTYPE_t d_ijk

    # ----------------------------------------------------------------------
    #  Initialize distance matrix
    #   - set diagonal to zero
    #   - symmetrize matrix if non-directed graph is desired
    dist_matrix.flat[::N + 1] = 0
    if not directed:
        for i in range(N):
            for j in range(i + 1, N):
                if dist_matrix[j, i] <= dist_matrix[i, j]:
                    dist_matrix[i, j] = dist_matrix[j, i]
                else:
                    dist_matrix[j, i] = dist_matrix[i, j]

    #----------------------------------------------------------------------
    #  Initialize predecessor matrix
    #   - check matrix size
    #   - initialize diagonal and all non-edges to NULL
    #   - initialize all edges to the row index
    cdef int store_predecessors = False

    if predecessor_matrix.size > 0:
        store_predecessors = True
        assert predecessor_matrix.shape[0] == N
        assert predecessor_matrix.shape[1] == N
        predecessor_matrix.fill(NULL_IDX)
        i_edge = np.where(~np.isinf(dist_matrix))
        predecessor_matrix[i_edge] = i_edge[0]
        predecessor_matrix.flat[::N + 1] = NULL_IDX

    # Now perform the Floyd-Warshall algorithm.
    # In each loop, this finds the shortest path from point i
    #  to point j using intermediate nodes 0 ... k
    if store_predecessors:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
                        predecessor_matrix[i, j] = predecessor_matrix[k, j]
    else:
        for k in range(N):
            for i in prange(N,nogil=True):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in prange(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk

In [122]:
%timeit floyd_warshall_scipy_parallel(adjlist)

1 loop, best of 3: 51.3 s per loop


In [10]:
#I took the source code of scipy, trimmed it down extensively, and put it here. 
#It took some work figuring out what to do with the types; they used a PXI file and included it originally
#Instead, I just put them at the top of the cell (this is the Dtype stuff)
#%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
%%cython -f
import warnings
from cython.parallel import prange
import cython.parallel as cp
import numpy as np
cimport numpy as np
cimport openmp
from scipy.sparse import csr_matrix, isspmatrix, isspmatrix_csr, isspmatrix_csc
from scipy.sparse.csgraph._validation import validate_graph

cimport cython

from libc.stdlib cimport malloc, free
from numpy.math cimport INFINITY

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

ITYPE = np.int32
ctypedef np.int32_t ITYPE_t

# Fused type for int32 and int64
ctypedef fused int32_or_int64:
    np.int32_t
    np.int64_t

# EPS is the precision of DTYPE
DEF DTYPE_EPS = 1E-15

# NULL_IDX is the index used in predecessor matrices to store a non-path
DEF NULL_IDX = -9999


class NegativeCycleError(Exception):
    def __init__(self, message=''):
        Exception.__init__(self, message)

def floyd_warshall_scipy_parallel_semir(csgraph, directed=True,
                   return_predecessors=False,
                   unweighted=False,
                   overwrite=False):
    dist_matrix = validate_graph(csgraph, directed, DTYPE,
                                 csr_output=False,
                                 copy_if_dense=not overwrite)
    if not isspmatrix(csgraph):
        # for dense array input, zero entries represent non-edge
        dist_matrix[dist_matrix == 0] = INFINITY

    if unweighted:
        dist_matrix[~np.isinf(dist_matrix)] = 1

    if return_predecessors:
        predecessor_matrix = np.empty(dist_matrix.shape,
                                      dtype=ITYPE, order='C')
    else:
        predecessor_matrix = np.empty((0, 0), dtype=ITYPE)

    _floyd_warshall(dist_matrix,
                    predecessor_matrix,
                    int(directed))

    if np.any(dist_matrix.diagonal() < 0):
        raise NegativeCycleError("Negative cycle in nodes %s"
                                 % np.where(dist_matrix.diagonal() < 0)[0])

    if return_predecessors:
        return dist_matrix, predecessor_matrix
    else:
        return dist_matrix

@cython.boundscheck(False)
cdef void _floyd_warshall(
               np.ndarray[DTYPE_t, ndim=2, mode='c'] dist_matrix,
               np.ndarray[ITYPE_t, ndim=2, mode='c'] predecessor_matrix,
               int directed=0):
    # dist_matrix : in/out
    #    on input, the graph
    #    on output, the matrix of shortest paths
    # dist_matrix should be a [N,N] matrix, such that dist_matrix[i, j]
    # is the distance from point i to point j.  Zero-distances imply that
    # the points are not connected.
    cdef int N = dist_matrix.shape[0]
    assert dist_matrix.shape[1] == N

    #cdef unsigned int i, j, k
    cdef int i, j, k
    cdef DTYPE_t d_ijk

    # ----------------------------------------------------------------------
    #  Initialize distance matrix
    #   - set diagonal to zero
    #   - symmetrize matrix if non-directed graph is desired
    dist_matrix.flat[::N + 1] = 0
    if not directed:
        for i in range(N):
            for j in range(i + 1, N):
                if dist_matrix[j, i] <= dist_matrix[i, j]:
                    dist_matrix[i, j] = dist_matrix[j, i]
                else:
                    dist_matrix[j, i] = dist_matrix[i, j]

    #----------------------------------------------------------------------
    #  Initialize predecessor matrix
    #   - check matrix size
    #   - initialize diagonal and all non-edges to NULL
    #   - initialize all edges to the row index
    cdef int store_predecessors = False

    if predecessor_matrix.size > 0:
        store_predecessors = True
        assert predecessor_matrix.shape[0] == N
        assert predecessor_matrix.shape[1] == N
        predecessor_matrix.fill(NULL_IDX)
        i_edge = np.where(~np.isinf(dist_matrix))
        predecessor_matrix[i_edge] = i_edge[0]
        predecessor_matrix.flat[::N + 1] = NULL_IDX

    # Now perform the Floyd-Warshall algorithm.
    # In each loop, this finds the shortest path from point i
    #  to point j using intermediate nodes 0 ... k
    if store_predecessors:
        for k in range(N):
            for i in range(N):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in range(N):
                    d_ijk = dist_matrix[i, k] + dist_matrix[k, j]
                    if d_ijk < dist_matrix[i, j]:
                        dist_matrix[i, j] = d_ijk
                        predecessor_matrix[i, j] = predecessor_matrix[k, j]
    else:
        for k in range(N):
            for i in prange(N,nogil=True):
                if dist_matrix[i, k] == INFINITY:
                    continue
                for j in prange(N):
                    dist_matrix[i,j]=min(dist_matrix[i,j],dist_matrix[i, k] + dist_matrix[k, j])

In [125]:
%timeit floyd_warshall_scipy_parallel_semir(adjlist)

1 loop, best of 3: 42 s per loop


In [11]:
#correctness checks
output1=floyd_warshall_scipy(adjlist)
output2=floyd_warshall_scipy_parallel(adjlist)
output3=floyd_warshall_scipy_parallel_semir(adjlist)

In [14]:
#yay we passed
print((output1==output2).all())
print((output3==output2).all())

True
True
